






























































































































































































































































Notebooks adaptados y traducidos a partir de los tutoriales de código abierto 
utilizados en las competencias virtuales de los últimos 2 años
Yamamoto, T., Terada, K., Ochiai, A. et al. Development of Human Support Robot as the research platform of a domestic mobile manipulator. Robomech J 6, 4 (2019). https://doi.org/10.1186/s40648-019-0132-3

Por favor Consulte los originales en inglés y Japonés en https://github.com/hsr-project/hsrb_robocup_dspl_docker


Adaptado al español por Laboratorio BioRobótica UNAM
Oscar Fuentes


Para correr este notebook debe tenerse familiaridad con términos de ROS como tópicos, listeners, publishers, mensajes etc.


En este notebook una simple maquina de estados para evasión de obstáculos

In [1]:
# Se importan las librerias necesarias
import math
import numpy as np
import rospy
import time
from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Twist


In [2]:
# A collection of conversion function for extracting numpy arrays from messages. http://wiki.ros.org/ros_numpy
import ros_numpy




In [3]:
print ('alive')

alive


In [4]:
# Iniciamos el nodo y se define el publisher de la velocidad a la que se movera la base del robot... Mensaje tipo twist
rospy.init_node('evasion_notebook_node') 
base_vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)

Nos valdremos de nuestra vieja función de movimiento, asi como la clase Laser para obtener las lecturas del sensor (revisar notebook 1)

In [5]:
def move_base_vel(vx, vy, vw):
    twist = Twist()
    twist.linear.x = vx
    twist.linear.y = vy
    twist.angular.z = vw 
    base_vel_pub.publish(twist)

def move_base(x,y,yaw,timeout=5):
    start_time = rospy.Time.now().to_sec()
    while rospy.Time.now().to_sec() - start_time < timeout:  
        move_base_vel(x, y, yaw)    

(state_machine.png "Maquina de estados")

Se definen las funciones ahora para mover al robot hacia adelante, hacia atras, giro a la izquierda 
y giro a la derecha, donde se indica una distancia y un angulo de rotacion fijo

In [30]:
def move_forward():
    move_base(0.15,0,0,1.0)
    stop()    
def move_backward():
    move_base(-0.15,0,0,0.5)
    stop()              
def turn_left():
    move_base(0.0,0,0.25*np.pi,2)
    stop()    
def turn_right():
    move_base(0.0,0,-0.25*np.pi,2)
    stop()    
def stop():
    move_base(0,0,0,0.1)

probamos nuestras funciones

In [31]:
move_forward()

In [32]:
stop()

In [33]:
move_backward()

In [34]:
turn_right()

se define la clase Laser, en la cual se lee la informacion del sensor, para esto se suscribe al topico ('/scan')

In [35]:
from sensor_msgs.msg import LaserScan

class Laser():
    u"""Class that handles laser information"""

    def __init__(self):
        # Register the _laser_cb method as a callback to the laser scan topic events
        self._laser_sub = rospy.Subscriber ('/scan',
                                           LaserScan, self._laser_cb)
        self._scan_data = None

    def _laser_cb (self, msg):
        # Laser scan callback function
        self._scan_data = msg

    def get_data(self):
        u"""Function to get the laser value"""
        return self._scan_data

In [36]:
laser = Laser()  #instanciamos una clase 

In [37]:
lectura=laser.get_data()
lectura
#len(lectura.ranges)/2

la siguiente funcion cuantiza los valores obtenidos por el sensor, retornando el valor de si que corresponde al sensor izquierdo
y sd para el derecho, un valor de cero si hay un obstaculo y de uno si se considera que no lo hay.

In [40]:
def get_lectura_cuant():
    lectura=np.asarray(laser.get_data().ranges) # convierte la info en arreglo numpy
    lectura=np.where(lectura>5,5,lectura) #remove infinito

    right_scan=lectura[int(3*len(lectura)/4):]   # discrimina por direccion
    left_scan=lectura[:int(len(lectura)/4)]
    front_scan=lectura[:30]

    sd,si=0,0
    if np.mean(left_scan)< 2.5: si=1 # promedia valores y cuantiza
    if np.mean(right_scan)< 2.5: sd=1
    print (np.mean(left_scan),np.mean(right_scan))
    return si,sd


In [41]:
si,sd=get_lectura_cuant()  # probamos la funcion
si,sd

2.944876692361302 3.653323533799913


(0, 0)

In [42]:
si,sd=get_lectura_cuant()
si,sd

2.9447436577743953 3.653478826416863


(0, 0)

A partir de aqui se define la maquina de estados

In [43]:
next_state= 's_0'
i=0
print (i, next_state, si, sd)

0 s_0 0 0


In [44]:
for i in range (202):## 20 pasos  pueden ser mas
    print ('step',i,'next ', next_state,'sensor izq', si,'sensor der', sd)
#print (next_state)
    if next_state== 's_0':
        print ('im in state ' , next_state)
        si,sd=get_lectura_cuant()
        #ESTADO 0 obtener lectura
        if (si==0 and sd==0): next_state='s_0'
            
        if (si==0 and sd==1): next_state='s_1'
        if (si==1 and sd==0): next_state='s_3'
        if (si==1 and sd==1): next_state='s_5'
        #####Accion
        move_forward()
        



    elif next_state== 's_1':
        print ('im in state 1')

        #####Accion
        #move_backward()
        

        next_state='s_2'


    elif next_state== 's_2':
        print ('im in state 2')
        #####Accion
        turn_left()
        next_state='s_0'


    elif next_state== 's_3':
        print ('im in state 3')
        #####Accion
        #move_backward()
        next_state='s_4'


    elif next_state== 's_4':
        print ('im in state 4')
        #####Accion
        turn_right()
        next_state='s_0'

    elif next_state== 's_5':
        print ('im in state 5')

        #####Accion

        #move_backward()
        next_state='s_6'

    elif next_state== 's_6':
        print ('im in state 6')
        #####Accion
        move_backward()
        next_state='s_7'

    elif next_state== 's_7':
        print ('im in state 7')
        #####Accion
        turn_left()
        next_state='s_8'

    elif next_state== 's_8':
        print ('im in state 8')

        #####Accion
        move_forward()
        next_state='s_9'

    elif next_state== 's_9':
        print ('im in state 9')
        #####Accion
        move_forward()
        next_state='s_10'

    elif next_state== 's_10':
        print ('im in state 10')
        #####Accion
        turn_right()
        next_state='s_11'
    elif next_state== 's_11':
        print ('im in state 11')

        #####Accion
        #turn_right()
        next_state='s_0'






step 0 next  s_0 sensor izq 0 sensor der 0
im in state  s_0
2.927733181582557 3.654413170284695
step 1 next  s_0 sensor izq 0 sensor der 0
im in state  s_0
3.2179721266031267 3.8154828733868067
step 2 next  s_0 sensor izq 0 sensor der 0
im in state  s_0
3.3588988953166536 3.9876050657696194
step 3 next  s_0 sensor izq 0 sensor der 0
im in state  s_0
3.252286203702291 4.065289950370788
step 4 next  s_0 sensor izq 0 sensor der 0
im in state  s_0
3.2416010869873895 4.237916254997254
step 5 next  s_0 sensor izq 0 sensor der 0
im in state  s_0
3.32338787317276 4.295243244700962
step 6 next  s_0 sensor izq 0 sensor der 0
im in state  s_0
3.4421932564841375 4.425867071416643
step 7 next  s_0 sensor izq 0 sensor der 0
im in state  s_0
2.811941831641727 3.9197223862012227
step 8 next  s_0 sensor izq 0 sensor der 0
im in state  s_0
2.773114589187834 3.877746084001329
step 9 next  s_0 sensor izq 0 sensor der 0
im in state  s_0
2.6313826587465075 3.833159073856142
step 10 next  s_0 sensor izq 0 se

KeyboardInterrupt: 